In [1]:
from collections import namedtuple

# Define a namedtuple
Person = namedtuple("Person", ["name", "age", "city"])

# Create an instance
p1 = Person(name="Alice", age=30, city="New York")

# Use _replace to create a modified version
p2 = p1._replace(age=31, city="San Francisco")

print(p1)  # Person(name='Alice', age=30, city='New York')
print(p2)  # Person(name='Alice', age=31, city='San Francisco')


Person(name='Alice', age=30, city='New York')
Person(name='Alice', age=31, city='San Francisco')
